### 3. Automatic Exam Creator

* OpenAI Use Case - completion API, can be used for text/story completion
* Prompt engineering - how to get certain results from the LLM

In [1]:
import os
import openai

In [2]:
# Import and pass api key
with open(r'C:\Users\mhuh22\Desktop\openai_api.txt', 'r') as file:
    api_key = file.readline().strip()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [11]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} "\
    +f"consisting of {num_questions} questions "\
    +f"Each question should have {num_possible_answers} options. "\
    +f"Also include the correct answer for each question "\
    +f"using the starting string 'Correct Answer: '"
    
    return prompt

In [19]:
prompt = create_test_prompt('US History',4,4)

In [20]:
# Send a query
response = openai.Completion.create(
    model = 'text-davinci-003',
    prompt = prompt,
    max_tokens = 256,
    temperature = 0.7
)

In [36]:
# Response comes out in json, print in readable format
test = response['choices'][0]['text']

In [26]:
def create_student_view(test, num_questions):
    student_view ={1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line + '\n'
        else:
            if question_number < num_questions:
                question_number += 1
                student_view[question_number] = ' '
    return student_view

In [30]:
# View only the questions
result = create_student_view(response['choices'][0]['text'],4)

for key in result:
    print(result[key])



Question 1: What year did the United States of America declare their independence?
A. 1762
B. 1776
C. 1789
D. 1812

 
Question 2: Who was the first president of the United States?
A. Thomas Jefferson
B. George Washington
C. Abraham Lincoln
D. John Adams

 
Question 3: When did the American Civil War begin?
A. 1861
B. 1864
C. 1868
D. 1877

 
Question 4: What amendment to the US Constitution abolished slavery?
A. Thirteenth Amendment
B. Fourteenth Amendment
C. Fifteenth Amendment
D. Sixteenth Amendment



In [55]:
test

'\n\nQuestion 1: What year did the United States of America declare their independence?\nA. 1762\nB. 1776\nC. 1789\nD. 1812\nCorrect Answer: B. 1776\n\nQuestion 2: Who was the first president of the United States?\nA. Thomas Jefferson\nB. George Washington\nC. Abraham Lincoln\nD. John Adams\nCorrect Answer: B. George Washington\n\nQuestion 3: When did the American Civil War begin?\nA. 1861\nB. 1864\nC. 1868\nD. 1877\nCorrect Answer: A. 1861\n\nQuestion 4: What amendment to the US Constitution abolished slavery?\nA. Thirteenth Amendment\nB. Fourteenth Amendment\nC. Fifteenth Amendment\nD. Sixteenth Amendment\nCorrect Answer: A. Thirteenth Amendment'

In [67]:
for line in test.split("\n"):
    if line.startswith("Correct Answer:"):
        print(line)

Correct Answer: B. 1776
Correct Answer: B. George Washington
Correct Answer: A. 1861
Correct Answer: A. Thirteenth Amendment


In [74]:
def extract_answer(test, num_questions):
    answers ={1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line + '\n'
        
            if question_number < num_questions:
                question_number += 1
                answers[question_number] = ' '
    return answers

In [76]:
extract_answer(response['choices'][0]['text'],4)

{1: 'Correct Answer: B. 1776\n',
 2: ' Correct Answer: B. George Washington\n',
 3: ' Correct Answer: A. 1861\n',
 4: ' Correct Answer: A. Thirteenth Amendment\n'}

In [37]:
student_view = create_student_view(test,4)

In [38]:
answers = extract_answer(test,4)

In [39]:
# Take the exam
def take(student_view):
    student_answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input('Enter your answer: ')
        student_answers[question] = answer
    return student_answers

In [47]:
student_answers = take(student_view)



Question 1: What year did the United States of America declare their independence?
A. 1762
B. 1776
C. 1789
D. 1812

Enter your answer: B
 
Question 2: Who was the first president of the United States?
A. Thomas Jefferson
B. George Washington
C. Abraham Lincoln
D. John Adams

Enter your answer: B
 
Question 3: When did the American Civil War begin?
A. 1861
B. 1864
C. 1868
D. 1877

Enter your answer: 1871
 
Question 4: What amendment to the US Constitution abolished slavery?
A. Thirteenth Amendment
B. Fourteenth Amendment
C. Fifteenth Amendment
D. Sixteenth Amendment

Enter your answer: A


In [53]:
answers[2]

' Correct Answer: B. George Washington\n'

In [51]:
student_answers

{1: 'B', 2: 'B', 3: '1871', 4: 'A'}

In [48]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question,answer in student_answers.items():
        if answer.upper() == correct_answer_dict[question][16]:
            correct_answers += 1
    
    grade = 100*correct_answers / len(answers)
    
    if grade < 60:
        passed = 'Fail'
    else:
        passed = 'Pass'
        
    return f"{correct_answers} / {len(answers)} correct! You got {grade} grade, {passed}"

In [49]:
grade(answers, student_answers)

'1 / 4 correct! You got 25.0 grade, Fail'